In [1]:
import numpy as np
import tensorflow as tf
import sys
sys.path.insert(1, '../libs')
from functions import *
from utils import *

In [7]:
STYLE_WEIGHTS = [3.0]
CONTENT_WEIGHTS = [1.0]
MODEL_SAVE_PATHS = ['../../models/style_weight_2e0.ckpt']

# for inferring (stylize)
INFERRING_CONTENT_DIR = '../../_inference/content'
INFERRING_STYLE_DIR = '../../_inference/style'
OUTPUTS_DIR = '../../_inference/output'

ENCODER_WEIGHTS_PATH = '../../vgg19_normalised.npz'



model_save_path = MODEL_SAVE_PATHS

content_imgs_path = list_images(INFERRING_CONTENT_DIR)
style_imgs_path   = list_images(INFERRING_STYLE_DIR)

In [9]:
contents_path = content_imgs_path
styles_path = style_imgs_path
output_dir = OUTPUTS_DIR
encoder_path = ENCODER_WEIGHTS_PATH
model_path = model_save_path
resize_height=None
resize_width=None

In [14]:
content = tf.placeholder(tf.float32, shape=(1, None, None, 3), name='content')
style   = tf.placeholder(tf.float32, shape=(1, None, None, 3), name='style')

In [22]:
tf.reshape(content, shape=tf.shape(style))

<tf.Tensor 'Reshape_6:0' shape=(1, ?, ?, 3) dtype=float32>

In [10]:
if isinstance(contents_path, str):
    contents_path = [contents_path]
if isinstance(styles_path, str):
    styles_path = [styles_path]

with tf.Graph().as_default(), tf.Session() as sess:
    # build the dataflow graph
    content = tf.placeholder(tf.float32, shape=(1, None, None, 3), name='content')
    style   = tf.placeholder(tf.float32, shape=(1, None, None, 3), name='style')

#         content = tf.placeholder(tf.float32, shape=INPUT_SHAPE, name='content')
#         style   = tf.placeholder(tf.float32, shape=INPUT_SHAPE, name='style')

    stn = STNet(encoder_path)

    output_image = stn.transform(content, style)

    sess.run(tf.global_variables_initializer())

    # restore the trained model and run the style transferring
    saver = tf.train.Saver()
    saver.restore(sess, model_path)

    outputs = []
    for content_path in contents_path:

        content_img = get_images(content_path, 
            height=resize_height, width=resize_width)

        for style_path in styles_path:

            style_img   = get_images(style_path)

            result = sess.run(output_image, 
                feed_dict={content: content_img, style: style_img})

            outputs.append(result[0])

save_images(outputs, contents_path, styles_path, output_dir, suffix=suffix)

NameError: name 'STNet' is not defined

In [ ]:
class SANet:
    '''
    Style-Attentional Network learns the mapping 
    between the content features and the style features 
    by slightly modifying the self-attention mechanism
    '''

    def __init__(self, num_filter):
        self.num_filter = num_filter

    # def map(self, content, style, scope='attention'):
    #     with tf.variable_scope(scope, reuse = tf.AUTO_REUSE):
    #         f = conv(content, self.num_filter // 8, kernel=1, stride=1, scope='f_conv') # [bs, h, w, c']
    #         g = conv(style,   self.num_filter // 8, kernel=1, stride=1, scope='g_conv') # [bs, h, w, c']
    #         h = conv(style,   self.num_filter     , kernel=1, stride=1, scope='h_conv') # [bs, h, w, c]

    #         # N = h * w
    #         s = tf.matmul(hw_flatten(g), hw_flatten(f), transpose_b=True) # # [bs, N, N]

    #         attention = tf.nn.softmax(s)  # attention map

    #         o = tf.matmul(attention, hw_flatten(h)) # [bs, N, C]
    #         # gamma = tf.get_variable("gamma", [1], initializer=tf.constant_initializer(0.0))

    #         o = tf.reshape(o, shape=content.shape) # [bs, h, w, C]
    #         o = conv(o, self.num_filter, kernel=1, stride=1, scope='attn_conv')

    #         # o = gamma * o + content
    #         o = o + content

    #         return o
    def map(self, content, style, scope='attention'):
        with tf.variable_scope(scope, reuse = tf.AUTO_REUSE):
            f = conv(content, self.num_filter // 8, kernel=1, stride=1, scope='f_conv') # [bs, h, w, c']
            g = conv(style,   self.num_filter // 8, kernel=1, stride=1, scope='g_conv') # [bs, h, w, c']
            h = conv(style,   self.num_filter     , kernel=1, stride=1, scope='h_conv') # [bs, h, w, c]

            s = tf.matmul(g, f, transpose_b=True)

            attention = tf.nn.softmax(s)

            o = tf.matmul(attention, h)
 
            # o = tf.reshape(o, shape=content.shape)
            o = conv(o, self.num_filter, kernel=1, stride=1, scope='attn_conv')

            o = o + content

            return o